In [4]:
# http://pbpython.com/amortization-model.html

# Payment, Principal and Interest

In [55]:
import pandas as pd
import numpy as np
from datetime import date

In [56]:
# Define the variables of the mortgage
Interest_rate = 0.04
Years = 30
Principal = 200000
Payments_per_year = 12 
Addl_Principal = 50 
start_date = (date(2018,1,1))

In [57]:
pmt = np.pmt(Interest_rate/Payments_per_year, Payments_per_year*Years,Principal)

In [58]:
pmt

-954.8305909309076

In [59]:
# This means that every month we need to pay $ 954.83. 

In [60]:
# We may also calculate principal payment and interest payment part for specific period, per

per = 1
ipmt = np.ipmt (Interest_rate/Payments_per_year,per,Payments_per_year*Years,Principal)
ppmt = np.ppmt(Interest_rate/Payments_per_year,per,Payments_per_year*Years,Principal)
print(ipmt,ppmt)

-666.6666666666667 -288.1639242642409


In [61]:
# Thus the first period payments is composed of $666.67 as interest part and $288.16 towards the principal part. 
# We can do a similar calculation for another period, for example for period 240 
per = 240 
ipmt = np.ipmt (Interest_rate/Payments_per_year,per,Payments_per_year*Years,Principal)
ppmt = np.ppmt(Interest_rate/Payments_per_year,per,Payments_per_year*Years,Principal)
print(ipmt,ppmt)

-316.49041533656924 -638.3401755943385


In [62]:
# What if we need to find the balance at the end of period 'x'. 

In [63]:
# Build a table 
rng = pd.date_range(start_date, periods = Years*Payments_per_year,freq = 'MS')
rng.name = "Payment_date"

In [64]:
rng

DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01',
               ...
               '2047-03-01', '2047-04-01', '2047-05-01', '2047-06-01',
               '2047-07-01', '2047-08-01', '2047-09-01', '2047-10-01',
               '2047-11-01', '2047-12-01'],
              dtype='datetime64[ns]', name='Payment_date', length=360, freq='MS')

In [65]:
# The function creates a range for next 30 years. The range will be used to build-up the basic data frame we will use in amortization 
# schdule. 
df = pd.DataFrame(index= rng, columns= ['Payment','Principal','Interest','Addl_Principal','Balance'],dtype = 'float')
df.head(5)

,Payment,Principal,Interest,Addl_Principal,Balance
Payment_date,,,,,
2018-01-01,NaN,NaN,NaN,NaN,NaN
2018-02-01,NaN,NaN,NaN,NaN,NaN
2018-03-01,NaN,NaN,NaN,NaN,NaN
2018-04-01,NaN,NaN,NaN,NaN,NaN
2018-05-01,NaN,NaN,NaN,NaN,NaN


In [66]:
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Period"

In [67]:
df.head(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance
Period,,,,,,
1,2018-01-01,NaN,NaN,NaN,NaN,NaN
2,2018-02-01,NaN,NaN,NaN,NaN,NaN
3,2018-03-01,NaN,NaN,NaN,NaN,NaN
4,2018-04-01,NaN,NaN,NaN,NaN,NaN
5,2018-05-01,NaN,NaN,NaN,NaN,NaN


In [68]:
# Now we can populate the data set with values of Payment, Principal and Interest
df["Payment"] = np.pmt(Interest_rate/Payments_per_year,Years*Payments_per_year,Principal)
df["Principal"] = np.ppmt(Interest_rate/Payments_per_year,df.index,Years*Payments_per_year,Principal)
df["Interest"] = np.ipmt(Interest_rate/Payments_per_year,df.index,Years*Payments_per_year,Principal)

In [69]:
df.head(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance
Period,,,,,,
1,2018-01-01,-954.830591,-288.163924,-666.666667,NaN,NaN
2,2018-02-01,-954.830591,-289.124471,-665.706120,NaN,NaN
3,2018-03-01,-954.830591,-290.088219,-664.742372,NaN,NaN
4,2018-04-01,-954.830591,-291.055180,-663.775411,NaN,NaN
5,2018-05-01,-954.830591,-292.025364,-662.805227,NaN,NaN


In [70]:
# We can add constant payment of $50 to Addl_Principal
df["Addl_Principal"] = -Addl_Principal
df = df.round(2)

In [71]:
df.head(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance
Period,,,,,,
1,2018-01-01,-954.83,-288.16,-666.67,-50,NaN
2,2018-02-01,-954.83,-289.12,-665.71,-50,NaN
3,2018-03-01,-954.83,-290.09,-664.74,-50,NaN
4,2018-04-01,-954.83,-291.06,-663.78,-50,NaN
5,2018-05-01,-954.83,-292.03,-662.81,-50,NaN


In [72]:
print(df.loc[1])

Payment_date      2018-01-01 00:00:00
Payment                       -954.83
Principal                     -288.16
Interest                      -666.67
Addl_Principal                    -50
Balance                           NaN
Name: 1, dtype: object


In [73]:
# We only need to update the last column, balance. There are two approaches that are shown below. 
# 1) using the loop method 

df["Balance"] = 0 
df.loc[1,"Balance"] = Principal + df.loc[1,"Principal"] + df.loc[1,"Addl_Principal"]
df.head(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance
Period,,,,,,
1,2018-01-01,-954.83,-288.16,-666.67,-50,199661.84
2,2018-02-01,-954.83,-289.12,-665.71,-50,0.00
3,2018-03-01,-954.83,-290.09,-664.74,-50,0.00
4,2018-04-01,-954.83,-291.06,-663.78,-50,0.00
5,2018-05-01,-954.83,-292.03,-662.81,-50,0.00


In [76]:
for i in range(2,len(df)+1):
    # Get the previous balance and current payments
    prev_balance = df.loc[i-1,"Balance"]
    principal = df.loc[i,"Principal"]
    addl_principal = df.loc[i,"Addl_Principal"]
    
    # If there is no balance, then zero out principal and balance
    if prev_balance == 0:
        df.loc[i,["Payment","Principal","Interest","Balance","Addl_Principal"]] = 0 
        continue
    
    # If this payment does not pay it off, then reduce the balance
    if abs(principal + addl_principal) <= prev_balance:
        df.loc[i,"Balance"] = principal + prev_balance + addl_principal
    
    # If this paymeny pays it off,zero out balance and adjust the final payment. 
    else: 
        # Adjust principal down 
        if prev_balance <= abs(principal):
            principal = - prev_balance
            addl_principal = 0 
        else:
            addl_principal = (prev_balance - abs(principal_payment))
        df.loc[i,"Balance"] = 0 
        df.loc[i,"Principal"] = principal 
        df.loc[i,"Addl_Principal"] = addl_principal  
        df.loc[i,"Payment"] = principal + df.ix[i,"Interest"]
df.round(2)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance
Period,,,,,,
1,2018-01-01,-954.83,-288.16,-666.67,-50,199661.84
2,2018-02-01,-954.83,-289.12,-665.71,-50,199322.72
3,2018-03-01,-954.83,-290.09,-664.74,-50,198982.63
4,2018-04-01,-954.83,-291.06,-663.78,-50,198641.57
5,2018-05-01,-954.83,-292.03,-662.81,-50,198299.54
6,2018-06-01,-954.83,-293.00,-661.83,-50,197956.54
7,2018-07-01,-954.83,-293.98,-660.86,-50,197612.56
8,2018-08-01,-954.83,-294.96,-659.88,-50,197267.60
9,2018-09-01,-954.83,-295.94,-658.89,-50,196921.66


In [83]:
# using Cumulative payments
df["Balance"] = 0
df["Cumulative Principal"] = (df["Principal"] + df["Addl_Principal"]).cumsum()

In [84]:
df.head(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance,Cumulative Principal
Period,,,,,,,
1,2018-01-01,-954.83,-288.16,-666.67,-50,0,-338.16
2,2018-02-01,-954.83,-289.12,-665.71,-50,0,-677.28
3,2018-03-01,-954.83,-290.09,-664.74,-50,0,-1017.37
4,2018-04-01,-954.83,-291.06,-663.78,-50,0,-1358.43
5,2018-05-01,-954.83,-292.03,-662.81,-50,0,-1700.46


In [86]:
df.tail(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance,Cumulative Principal
Period,,,,,,,
356,2047-08-01,0.0,0.0,0.0,0,0,-200000.0
357,2047-09-01,0.0,0.0,0.0,0,0,-200000.0
358,2047-10-01,0.0,0.0,0.0,0,0,-200000.0
359,2047-11-01,0.0,0.0,0.0,0,0,-200000.0
360,2047-12-01,0.0,0.0,0.0,0,0,-200000.0


In [92]:
df["Cumulative Principal"] = df["Cumulative Principal"].clip(lower = - Principal)

In [97]:
df = df.drop(["Cumulative Pricipal"],axis=1)

In [98]:
df.tail(5)

,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance,Cumulative Principal
Period,,,,,,,
356,2047-08-01,0.0,0.0,0.0,0,3.492460e-10,-200000.0
357,2047-09-01,0.0,0.0,0.0,0,3.492460e-10,-200000.0
358,2047-10-01,0.0,0.0,0.0,0,3.492460e-10,-200000.0
359,2047-11-01,0.0,0.0,0.0,0,3.492460e-10,-200000.0
360,2047-12-01,0.0,0.0,0.0,0,3.492460e-10,-200000.0


In [99]:
df["Balance"] = Principal + df["Cumulative Principal"]

In [105]:
df.head(5)
df = df.round(2)

In [104]:
df.tail(30)


,Payment_date,Payment,Principal,Interest,Addl_Principal,Balance,Cumulative Principal
Period,,,,,,,
331,2045-07-01,-954.83,-864.11,-90.72,-50,9.802010e+03,-190197.99
332,2045-08-01,-954.83,-866.99,-87.84,-50,8.885020e+03,-191114.98
333,2045-09-01,-954.83,-869.88,-84.95,-50,7.965140e+03,-192034.86
334,2045-10-01,-954.83,-872.78,-82.05,-50,7.042360e+03,-192957.64
335,2045-11-01,-954.83,-875.69,-79.14,-50,6.116670e+03,-193883.33
336,2045-12-01,-954.83,-878.61,-76.22,-50,5.188060e+03,-194811.94
337,2046-01-01,-954.83,-881.54,-73.29,-50,4.256520e+03,-195743.48
338,2046-02-01,-954.83,-884.48,-70.36,-50,3.322040e+03,-196677.96
339,2046-03-01,-954.83,-887.42,-67.41,-50,2.384620e+03,-197615.38


In [107]:
# Let us find the installment which corresponds to last payment. 
last_payment = df.query("Cumulative Principal <= -200000")["Cumulative Principal"].idxmax(axis=1,skipna=True)

SyntaxError: invalid syntax (<unknown>, line 1)